In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
import get_data
df, descriptor_names, key_name = get_data.load("Fe2")

ans_list = df[key_name].values

In [ ]:
METADATA = {"outputdir": "image_executed", "prefix": "clustering_Fe2", 
              "dataname": "Fe2"}

In [ ]:
from sklearn.preprocessing import StandardScaler
Xraw = df[descriptor_names].values
scaler = StandardScaler()
scaler.fit(Xraw)
X = scaler.transform(Xraw)


In [ ]:
from sklearn.cluster import KMeans
nclusters = 3
km = KMeans(nclusters)
km.fit(X)
yp_km = km.predict(X)


In [ ]:
from sklearn.decomposition import PCA
drd = PCA(2)
drd.fit(X)
X_PCA = drd.transform(X)


In [ ]:
from clustering_misc import plot_X2_ystring
plot_X2_ystring(X_PCA, ans_list, yp_km, comment="km", metadata=METADATA)


In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(nclusters)
gmm.fit(X)
yp_gmm = gmm.predict(X)


In [ ]:
plot_X2_ystring(X_PCA, ans_list, yp_gmm, comment="gmm", metadata=METADATA)


In [ ]:
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
metric = 'euclidean'
pairdistance = pdist(X, metric=metric)
method = 'single'
Z = linkage(pairdistance, method=method)


In [ ]:
from scipy.cluster.hierarchy import dendrogram
import os
fig, ax = plt.subplots(figsize=(5, 10))
label_list = df["key"].values
tree = dendrogram(Z, labels=label_list, orientation="left", ax=ax)
ax.invert_yaxis()
fig.tight_layout()
filename = "_".join([METADATA["prefix"], "dendrogram"])+".png"
print(filename)
plt.savefig(os.path.join(METADATA["outputdir"], filename))

In [ ]:
from scipy.spatial.distance import squareform
corr = 1- np.abs(df[descriptor_names].corr()) 
pairdistance = squareform(corr)

Z = linkage(pairdistance)
fig, ax = plt.subplots(figsize=(5,3))
tree = dendrogram(Z, labels=descriptor_names, orientation="left", ax=ax)
filename = "_".join([METADATA["prefix"], "descriptor","dendrogram"])+".png"
print(filename)
fig.tight_layout()
fig.savefig(os.path.join(METADATA["outputdir"], filename))